In [ ]:
###################### Magic commands

##%%timeit -n 1 -r 1
#%%time
import time
#start = time.time()

###################### at the end

#end = time.time()
#print(end - start)

###################### Packages

import numpy as np
import numpy.random as npr
import pandas as pd
import scipy.special
import math 
import numba
import matplotlib.pyplot as plt

# pip install fbm
# from fbm import FBM

###################### Options

npr.seed(0)
import warnings
warnings.filterwarnings('ignore')

#<
#>

############### Tools transfering numpy to numba

@numba.njit(parallel=True, fastmath=False) 
def np_apply_along_axis_2darray(func1d, axis, arr):
  assert arr.ndim == 2
  assert axis in [0, 1]
  if axis == 0:
    result = np.empty(arr.shape[1])
    for i in numba.prange(len(result)):
      result[i] = func1d(arr[:, i])
  else:
    result = np.empty(arr.shape[0])
    for i in numba.prange(len(result)):
      result[i] = func1d(arr[i, :])
  return result

@numba.njit(parallel=True, fastmath=False) 
def np_mean_2darray(array, axis):
  return np_apply_along_axis(np.mean, axis, array)

@numba.njit(parallel=True, fastmath=False) 
def np_std_2darray(array, axis):
  return np_apply_along_axis(np.std, axis, array)

################################################## Non-empty

def check_cond(gamma,c,tau,q):
    if 0<tau:
        if gamma<1 and 0<gamma and 2<q and 0<c and 2*(c+tau)*gamma<1 and q*(1-2*tau*gamma)>2 and q*c*gamma>1 and tau <1/(2*gamma):
            print('Valid!')
        else:
            print('Not valid!')
    else:
        if gamma<1 and 0<gamma and 2<q and 0<c and 2*(c+tau)*gamma<1 and q*c*gamma>1:
            print('Valid!')
        else:
            print('Not valid!')        
    return

def check_cond_no_q(gamma,c,tau):
    if gamma<1 and 0<gamma and 0<c and 2*(c+tau)*gamma<1:
        print('Valid!')
    else:
        print('Not valid!')   
    return

@numba.njit(parallel=True, fastmath=True)
def find_cond_all(h_gamma,h_c,h_tau,h_q,max_c,max_tau,max_q):
    gamma_mesh = np.linspace(0.01,0.99,h_gamma)
    c_mesh = np.linspace(0.1,max_c,h_c)
    tau_mesh = np.linspace(-max_tau,max_tau,h_tau)
    q_mesh = np.linspace(2.1,max_q,h_q)
    for gamma_index in numba.prange(h_gamma):
        for c_index in numba.prange(h_c): 
            for tau_index in numba.prange(h_tau):
                for q_index in numba.prange(h_q):
                    gamma=gamma_mesh[gamma_index]
                    c=c_mesh[c_index]
                    tau=tau_mesh[tau_index]
                    q=q_mesh[q_index]
                    if 0<tau:
                        if 2<q and 0<c and 2*(c+tau)*gamma<1 and q*(1-2*tau*gamma)>2 and q*c*gamma>1:
                            print([gamma, c, tau, q])
                    else:
                        if 2<q and 0<c and 2*(c+tau)*gamma<1 and q*c*gamma>1:
                            print([gamma, c, tau, q])                        
    return 'Done!'

@numba.njit(parallel=True, fastmath=False)
def find_cond_no_q(grid_gamma,max_tau):
    gamma_mesh = np.linspace(0,1,grid_gamma+1)[1:grid_gamma]
    c_mesh = np.arange(1,10)
    tau_mesh = np.arange(-max_tau,max_tau+1)
    for gamma_index in numba.prange(gamma_mesh.shape[0]):
        for c_index in numba.prange(c_mesh.shape[0]): 
            for tau_index in numba.prange(tau_mesh.shape[0]):
                gamma=gamma_mesh[gamma_index]
                c=c_mesh[c_index]
                tau=tau_mesh[tau_index]
                if 2*(c+tau)*gamma<1:
                    print([gamma, c, tau])
    return 'Done!'

@numba.njit(parallel=True, fastmath=False)
def find_gamma_no_q(grid_gamma,c,tau):
    gamma_mesh = np.linspace(0,1,grid_gamma+1)[1:grid_gamma]
    for gamma_index in numba.prange(gamma_mesh.shape[0]):
        gamma=gamma_mesh[gamma_index]
        if 2*(c+tau)*gamma<1:
            print([gamma, c, tau])
    return 'Done!'

@numba.njit(parallel=True, fastmath=False)
def find_cond_c_no_q(c,grid_gamma,max_tau):
    gamma_mesh = np.linspace(0,1,grid_gamma+1)[1:grid_gamma]
    tau_mesh = np.arange(-max_tau,max_tau+1)
    for gamma_index in numba.prange(gamma_mesh.shape[0]):
        for tau_index in numba.prange(tau_mesh.shape[0]):
            gamma=gamma_mesh[gamma_index]
            tau=tau_mesh[tau_index]
            if 2*(c+tau)*gamma<1:
                print([gamma, c, tau])
    return 'Done!'

@numba.njit(parallel=True, fastmath=False)
def find_cond_c_tau_no_q(c,tau,grid_gamma):
    gamma_mesh = np.linspace(0,1,grid_gamma+1)[1:grid_gamma]
    tau_mesh = np.arange(-max_tau,max_tau+1)
    for gamma_index in numba.prange(gamma_mesh.shape[0]):
        for tau_index in numba.prange(tau_mesh.shape[0]):
            gamma=gamma_mesh[gamma_index]
            tau=tau_mesh[tau_index]
            if 2*(c+tau)*gamma<1:
                print([gamma, c, tau])
    return 'Done!'

################################################## Generating Data

@numba.njit(parallel=False, fastmath=False)
def Lomax_quantile_function(x,theta,s):  
    return s*((1-x)**(-1/theta)-1)

@numba.njit(parallel=True, fastmath=False)
def Pareto_quantile_function(x,gamma,s): # support is  {x \ge s} and theta=1/\gamma
    return s*(1-x)**(-gamma)

#The Burr distribution has survival distribution $\bar{F}(y)=(1+y^\rho)^{-\theta} \in 2\RV_{-\theta \rho,-\rho}$ where $x\ge 0$ and $\theta,\rho>0$.
@numba.njit(parallel=True, fastmath=False) # rho,theta positive
def Burr_quantile_function(x,theta,rho): 
    return ((1-x)**(-1/theta)-1)**(1/rho)

@numba.njit(parallel=True, fastmath=False)
def beta_func3(d):
    grid=np.linspace(0,1,d)
    norm=np.sqrt(np.sum(np.exp(-grid**2+grid)**2)/d)
    return np.exp(-grid**2+grid)/norm

@numba.njit(parallel=True, fastmath=False)
def beta_func2(d):
    grid=np.linspace(0,1,d)
    norm=np.sqrt(np.sum(np.exp(-grid)**2)/d)
    return np.exp(-grid)/norm

@numba.njit(parallel=True, fastmath=False)
def beta_func(d):
    grid=np.linspace(0,1,d)
    norm=np.sqrt(np.sum(np.sin(2*np.pi*grid)**2)/d)
    return np.sin(2*np.pi*grid)/norm


@numba.njit(parallel=True, fastmath=False)
def coeurjolly_cholesky_fbm_1D(d,H,sigma):
    H2 = 2 * H
    matcov = np.zeros((d-1,d-1))
    for i in numba.prange(d-1):
        for j in numba.prange(i,d-1):
            r = (sigma**2)*(1/2)*(abs(i+1)**H2 + abs(j+1)**H2 - abs(j - i)**H2)
            r = r/(d**H2)
            matcov[i, j] = r
            matcov[j, i] = matcov[i, j]
    L = np.linalg.cholesky(matcov)
    Z = npr.normal(0,1,size=(d - 1))
    fBm = np.dot(L , Z)
    #out=np.concatenate(([0], fBm))
    # out=np.hstack(([0], fBm))
    out= np.asarray([0] + list(fBm))
    return out

# @Article{RePEc:jss:jstsof:v:005:i07,
#  author={Coeurjolly, Jean-Francois},
#  title={{Simulation and identification of the fractional Brownian motion: a bibliographical and comparative study}},
#  journal={Journal of Statistical Software},
#  year=2000,
#  volume={5},
#  number={i07},
#  pages={},
#  month={},
#  keywords={},
#  doi={http://hdl.handle.net/10.18637/jss.v005.i07}
#}

@numba.njit(parallel=True, fastmath=False)
def coeurjolly_cholesky_fbm_array(Z,H,sigma): # Z=npr.normal(0,1,size=(N,n,d - 1))
    N=Z.shape[0]
    n=Z.shape[1]
    d=Z.shape[2]+1
    out = np.zeros((N,n,d))
    for p in numba.prange(N):
        for q in numba.prange(n):    
            H2 = 2 * H
            matcov = np.zeros((d-1,d-1))
            for i in numba.prange(d-1):
                for j in numba.prange(i,d-1):
                    r = (sigma**2)*(1/2)*(abs(i+1)**H2 + abs(j+1)**H2 - abs(j - i)**H2)
                    r = r/(d**H2)
                    matcov[i, j] = r
                    matcov[j, i] = matcov[i, j]
            L = np.linalg.cholesky(matcov)
            fBm = np.dot(L , Z[p,q,:])
            #out=np.concatenate(([0], fBm))
            # out=np.hstack(([0], fBm))
            out[p,q,:]= np.asarray([0] + list(fBm))
    return out

@numba.njit(parallel=True, fastmath=False)
def sigma(u,c,snr): 
    return (u**c)/snr

@numba.njit(parallel=True, fastmath=False)
def noise_mean(d,mu):
    grid=np.linspace(0,1,d)
    return mu*grid

@numba.njit(parallel=True, fastmath=False)
def coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu): #Z = npr.normal(0,1,size=(N,n,d - 1))  
                                     #Y = Pareto_iterated_sample(N,n,theta,s)
    N=Y.shape[0]
    n=Y.shape[1]
    d=Z.shape[2]+1
    out = np.zeros((N,n,d))
    H2 = 2 * H
    matcov = np.zeros((d-1,d-1))
    for p in numba.prange(N):
        for q in numba.prange(n):
            matcov = np.zeros((d-1,d-1))
            for i in numba.prange(d-1):
                for j in numba.prange(i,d-1):
                    r = (sigma(Y[p,q],c,snr)**2)*(1/2)*(abs(i+1)**H2 + abs(j+1)**H2 - abs(j - i)**H2)
                    r = r/(d**H2)
                    matcov[i, j] = r
                    matcov[j, i] = matcov[i, j]
            L = np.linalg.cholesky(matcov)
            fBm = np.dot(L , Z[p,q,:])
            out[p,q,:]= np.asarray([0]+list(fBm)) + noise_mean(d,mu)
    return out

################################################## Estimation

@numba.njit(parallel=True, fastmath=False)
def threshold(X,Y,Y_sort_index,tau,m): # 1\le k \le n
    N=X.shape[0]
    n=X.shape[1]
    y_matrix_out = np.zeros((N,n))
    aux=concomittant_corr(X,Y,Y_sort_index,tau,int(n/5))[:,4:]
    YY=np.copy(Y)
    Y_sort=sort_2d_array(YY)
    for i in numba.prange(N):
        index = np.argmax(aux[i,:])
        y_matrix_out[i,:] = Y_sort[i,n-index-1]
    return y_matrix_out

@numba.njit(parallel=True, fastmath=False)
def threshold_index(X,Y,Y_sort_index,tau,m): # 1\le k \le n
    N=X.shape[0]
    n=X.shape[1]
    out=np.zeros((N,))
    aux=concomittant_corr(X,Y,Y_sort_index,tau,int(n/5))[:,4:]
    for i in numba.prange(N):
        out[i] = np.argmax(aux[i,:])
    return out

#@numba.njit(parallel=True, fastmath=False) # It seems that "greater_equal" and numba don't work well together
def fepls(X,Y,y_matrix,tau): # X of size (N,n,d) and Y of size (N,n)
                             # y_matrix of shape (N,n), for instance y_matrix = y*np.ones((N,n)) where y threshold
                             # tau is the tail index of \vfi
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]
    out=np.zeros((N,d))
    for j in range(d):
        aux = np.multiply(X[:,:,j],Y**tau) # size (N,n,d) - Product \vfi(Y_i)*X_i
        out2 = np.multiply(aux,np.greater_equal(Y,y_matrix)) # size (N,n) - Product \vfi(Y_i)*X_i*1_{Y_i \ge y}
        out[:,j]= np.sum(out2,axis=1)/n # (N,d)
    norms=np.sqrt(np.sum(out**2,axis=1)/d) # length (N,)
    out2 =  out * (norms.reshape((norms.size, 1)))**(-1)
    return out2 # size (N,d)

@numba.njit(parallel=True, fastmath=False) 
def fepls_numba(X,Y,y_matrix,tau): # X of size (N,n,d) and Y of size (N,n)
                             # y_matrix of shape (N,n), for instance y_matrix = y*np.ones((N,n)) where y threshold
                             # tau is the tail index of \vfi
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]
    out=np.zeros((N,d))
    for j in numba.prange(d):
        aux = np.multiply(X[:,:,j],Y**tau) # size (N,n,d) - Product \vfi(Y_i)*X_i
        out2 = np.multiply(aux,np.greater_equal(Y,y_matrix)) # size (N,n) - Product \vfi(Y_i)*X_i*1_{Y_i \ge y}
        out[:,j]= np.sum(out2,axis=1)/n # (N,d)
    norms=np.sqrt(np.sum(out**2,axis=1)/d) # length (N,)
    out2 =  out * (norms.reshape((norms.size, 1)))**(-1)
    return out2 # size (N,d)



# g(t)= t^c with c \in \{1/4,1/2,1,3/2\} 
# X = g(Y)\beta + \eps

@numba.njit(parallel=True, fastmath=False) 
def Hill(Y,int_seq): # Y of size (N,n), y number, int_seq is an intermediate sequence, ie such that int_seq << n
    N=Y.shape[0]
    n=Y.shape[1]
    Y_ord=np.copy(Y)
    Y_ord=np.sort(Y_ord) 
    Y_2=Y_ord[:,n-int_seq-1]
    aux=Y_ord/Y_2[:, None]
    out=np.log(aux[:,0:n-int_seq])
    return (1/int_seq)*np.sum(out,axis=1) # size (N,d)

#### Same as np.sort for 2D arrays but works with numba njit+parallel
@numba.njit(parallel=True, fastmath=False) 
def sort_2d_array(x):
    n,m=np.shape(x)
    for row in numba.prange(n):
        x[row]=np.sort(x[row])
    return x

@numba.njit(parallel=True, fastmath=False) 
def hatbeta_dot_beta(X,Y,tau,l):
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]    
    y_array=np.zeros((N,n,np.arange(int(n/l)).size))
    out=np.zeros((N,np.arange(int(n/l)).size))
    YY=np.copy(Y)
    for p in numba.prange(int(n/l)):
        y_array[:,0,p]=sort_2d_array(YY)[:,n-l*p-1]
        for k in numba.prange(N):
            y_array[k,:,p]=y_array[k,0,p]
        hat_beta=fepls_numba(X,Y,y_array[:,:,p],tau) 
        out[:,p]=(1/d)*np.sum(np.multiply(hat_beta,X[:,p,:]),axis=1) 
    return out

@numba.njit(parallel=True, fastmath=False) 
def concomittant_corr(X,Y,Y_sort_index,tau,m): # 1\le m \le n # Y_sort_index = np.argsort(Y,axis=1)
    N = X.shape[0]
    n = X.shape[1]
    d = X.shape[2]
    out = np.zeros((N,m))
    YY=np.copy(Y)
    Y_sort=sort_2d_array(YY)
    for k in numba.prange(m):
        y_array = np.zeros((N,n,k+1))
        aux = np.zeros((N,k+1))
        aux2 = np.zeros((N,k+1))
        aux3 = Y_sort[:,n-k-1:] # shape (N,k+1)
        aux3_sum = np.sum(aux3,axis=1)
        for i in numba.prange(k):
            y_array[:,0,i] = Y_sort[:,n-i-1]
            for j_2 in numba.prange(N):
                y_array[j_2,:,i] = y_array[j_2,0,i]
            hat_beta = fepls_numba(X,Y,y_array[:,:,i],tau) 
            for j_1 in numba.prange(N):
                i_c = Y_sort_index[j_1,i]
                aux[j_1,i]=(1/d)*np.sum(np.multiply(hat_beta[j_1,:],X[j_1,i_c,:]))
                aux2[j_1,i]= np.multiply(aux[j_1,i],Y_sort[j_1,n-i-1]) 
                out[j_1,k]= np.corrcoef(aux3[j_1,:],aux[j_1,:])[0,1]
    return out
    
@numba.njit(parallel=True, fastmath=False) 
def hatbeta_dot_X(X,hat_beta,y_matrix,tau): # hat_beta=fepls(X,Y,y_matrix,tau) of shape (N,d)
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]    
    out=np.zeros((N,n))
    for i in numba.prange(n):
            out[:,i]=(1/d)*np.sum(np.multiply(hat_beta,X[:,i,:]),axis=1) 
    return out

@numba.njit(parallel=True, fastmath=False) 
def beta_dot_X(X):
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]    
    out=np.zeros((N,n))
    for i in numba.prange(n):
            out[:,i]=(1/d)*np.sum(np.multiply(X[:,i,:],beta_func(d)),axis=1) 
    return out

@numba.njit(parallel=True, fastmath=False) 
def conditional_cov_Y_hat_beta_X(X,Y,y_matrix,tau): #hat_beta = fepls(X,Y,y_matrix,tau)
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]    
    A=hatbeta_dot_X(X,hat_beta,y_matrix,tau)
    cov_A=np.zeros((N,))
    cov_B=np.zeros((N,))
    for k in numba.prange(N):
        cov_A[k] = (((A[k,:])[Y[k,:]>y])*((Y[k,:])[Y[k,:]>y])).mean()-((A[k,:])[Y[k,:]>y]).mean()*((Y[k,:])[Y[k,:]>y]).mean()
        #cov_B[k] = (((B[k,:])[Y[k,:]>y])*((Y[k,:])[Y[k,:]>y])).mean()-((B[k,:])[Y[k,:]>y]).mean()*((Y[k,:])[Y[k,:]>y]).mean()
    return cov_A

#@numba.njit(parallel=True, fastmath=False) # does not work with numba (no idea why)
def conditional_cov_Y_beta_X(X,Y,y):
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]    
    B=beta_dot_X(X)
    cov_B=np.zeros((N,))
    for k in numba.prange(N):
        cov_B[k] = (((B[k,:])[Y[k,:]>y])*((Y[k,:])[Y[k,:]>y])).mean()-((B[k,:])[Y[k,:]>y]).mean()*((Y[k,:])[Y[k,:]>y]).mean()
    return cov_B

################################################## Execute

N=500
n=500
d=101
s_Y=1 # scale parameter of Pareto/Lomax distribution
c= 1/2 # c is \kappa the tail index of g
tau=-2 # tail index of \vfi
#nu = 1 #  tail index of \psi such that 2\gamma \nu_j < 1 for all 1\le j \le J
snr=10 # signal-to-noise ratio
H=1/3 # Hurst parameter of fBm noise
gamma=9/10# 1/3 or 1/2 or 9/10
rho=-1/2
s = 3/4 # tunes the threshold y_n (see below)
l=2 # grid parameter
#y=n**(s*gamma) # threshold y_n
#y_matrix = y*np.ones((N,n)) # threshold matrix
mu = 200 # noise mean

tic=time.time()

#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)

Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps

#############################################################################
print(check_cond_no_q(gamma,c,tau))
print("Time cost",time.time()-tic)
##################################################################################################

In [ ]:
################## Histogram 

Y_sort_index = np.argsort(Y,axis=1)
m=int(n/5)
y_matrix = threshold(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix[:,0], bins=m)
plt.show()

In [ ]:
############################################################################# Estim plot

Y_sort_index = np.argsort(Y,axis=1)
m=int(n/5)
y_matrix = threshold(X,Y,Y_sort_index,tau,m)

E = fepls(X,Y,y_matrix,tau)

# Calculate the maximum and minimum values for each position
#max_values = np.max(E, axis=0) # shape (d,)
#min_values = np.min(E, axis=0) # shape (d,)
max_values = np.nanquantile(E, 0.95, axis=0) # shape (d,)
min_values = np.nanquantile(E, 0.05, axis=0) # shape (d,)
mean_values = np.nanmean(E, axis=0) # shape (d,)
median_values = np.nanmedian(E, axis=0) # shape (d,)

# Create x values (assuming x values are just indices in this case)
x_values = np.arange(d)

fig,ax = plt.subplots()

# Set x-axis tick labels to discretized interval [0,1]
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
ax.set_xticklabels([0,0 , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1 ])

# Plot the fill between the max and min values
ax.fill_between(x_values, min_values, max_values, color='skyblue', alpha=0.4)
ax.plot(x_values, mean_values)
ax.plot(x_values, beta_func(d))

# Add labels and legend
#ax.xlabel('Index')
#ax.ylabel('Values')
#ax.title('FEPLS Estimation of beta - Confidence intervals')
#plt.legend()
# Show the plot
plt.savefig('beta_estim_plot_conc100_2_0_9.pdf')  
plt.show()

In [ ]:
######################################## Comparison tail index estimation with X and with <X,hat_beta>
tic=time.time()
alpha=0.8
J=9
h=mu
h_vect=h*np.ones((N,))

plot1=plot_tail_index_func(X,Y,h_vect,c,gamma,rho,alpha,J)

hat_beta=fepls(X,Y,y_matrix,tau)
X_dimred = hatbeta_dot_X(X,hat_beta,y_matrix,tau)
plot2=plot_tail_index_uni(X_dimred,Y,h_vect,c,gamma,rho,alpha,J)

max_values1 = np.nanquantile(plot1, 0.95, axis=0) # shape (d,)
min_values1 = np.nanquantile(plot1, 0.05, axis=0) # shape (d,)
mean_values1 = np.nanmean(plot1, axis=0) # shape (d,)
median_values1 = np.nanmedian(plot1, axis=0) # shape (d,)

max_values2 = np.nanquantile(plot2, 0.95, axis=0) # shape (d,)
min_values2 = np.nanquantile(plot2, 0.05, axis=0) # shape (d,)
mean_values2 = np.nanmean(plot2, axis=0) # shape (d,)
median_values2 = np.nanmedian(plot2, axis=0) # shape (d,)

# Create x values (assuming x values are just indices in this case)
x_values = np.arange(d)

fig,ax = plt.subplots()

# Set x-axis tick labels to discretized interval [0,1]
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
ax.set_xticklabels([0,0 , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1 ])

# Plot the fill between the max and min values
ax.fill_between(x_values, min_values1, max_values1, color='skyblue', alpha=0.5)
ax.plot(x_values, mean_values1,color='blue')
ax.fill_between(x_values, min_values2, max_values2, color='yellow', alpha=0.5)
ax.plot(x_values, mean_values2,color='orange')
ax.hlines(gamma,xmin=0,xmax=100,linewidth=1, color='r')

plt.show()
print("Time cost",time.time()-tic)


In [ ]:
####################################################### hatbeta_dot_beta
A=hatbeta_dot_beta(X,Y,-2,l)
A2=hatbeta_dot_beta(X,Y,-3,l)
A3=hatbeta_dot_beta(X,Y,-4,l)


# Calculate the maximum and minimum values for each position
mean_values_A = np.nanmean(A, axis=0) # shape (int(n/l),)
#median_values_A = np.nanmedian(A, axis=0) # shape (int(n/l),)

mean_values_A2 = np.nanmean(A2, axis=0) # shape (int(n/l),)
#median_values_A2 = np.nanmedian(A2, axis=0) # shape (int(n/l),)

mean_values_A3 = np.nanmean(A3, axis=0) # shape (int(n/l),)
#median_values_A3 = np.nanmedian(A3, axis=0) # shape (int(n/l),)

# Create x values (assuming x values are just indices in this case)
x_values_A = np.arange(int(n/l))

fig,ax = plt.subplots()

# Plot the fill between the max and min values
#ax.fill_between(x_values_A, min_values_A, max_values_A, color='skyblue', alpha=0.4, label='Confidence Area')
ax.plot(x_values_A, mean_values_A, label='tau = -2')

#ax.fill_between(x_values_A, min_values_A2, max_values_A2, color='skyblue', alpha=0.4, label='Confidence Area')
ax.plot(x_values_A, mean_values_A2, label='tau = -3')

#ax.fill_between(x_values_A, min_values_A3, max_values_A3, color='skyblue', alpha=0.4, label='Confidence Area')
ax.plot(x_values_A, mean_values_A3, label='tau = -4')



# Add labels and legend
#ax.xlabel('Index')
#ax.ylabel('Values')
#ax.title('FEPLS Estimation of beta - Confidence intervals')
plt.legend()
plt.savefig('beta_estim_exceedance_2_0_9.pdf')
# Show the plot
plt.show()



In [ ]:
#################################################### Correlation (concomittant) plot

m=int(n/2) # 1\le m \le n

A=concomittant_corr(X,Y,Y_sort_index,-1,m)[:,1:]
A2=concomittant_corr(X,Y,Y_sort_index,-2,m)[:,1:]
A3=concomittant_corr(X,Y,Y_sort_index,-3,m)[:,1:]


# Calculate the maximum and minimum values for each position

mean_values_A = np.nanmean(A, axis=0) # shape (int(n/l),)
#median_values_A = np.nanmedian(A, axis=0) # shape (int(n/l),)

mean_values_A2 = np.nanmean(A2, axis=0) # shape (int(n/l),)
#median_values_A2 = np.nanmedian(A2, axis=0) # shape (int(n/l),)

mean_values_A3 = np.nanmean(A3, axis=0) # shape (int(n/l),)
#median_values_A3 = np.nanmedian(A3, axis=0) # shape (int(n/l),)

# Create x values (assuming x values are just indices in this case)
x_values_A = np.arange(m-1)#np.arange(int(n/l))

fig,ax = plt.subplots()

# Set x-axis tick labels to discretized interval [1,100]
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
ax.set_xticklabels([1, 1, 10 , 20, 30,  40, 50, 60, 70, 80, 90, 100 ])

# Plot the fill between the max and min values
#ax.fill_between(x_values_A, min_values_A, max_values_A, color='skyblue', alpha=0.4, label='Confidence Area')
ax.plot(x_values_A, mean_values_A, label='tau = -1')

#ax.fill_between(x_values_A, min_values_A2, max_values_A2, color='skyblue', alpha=0.4, label='Confidence Area')
ax.plot(x_values_A, mean_values_A2, label='tau = -2')

#ax.fill_between(x_values_A, min_values_A3, max_values_A3, color='skyblue', alpha=0.4, label='Confidence Area')
ax.plot(x_values_A, mean_values_A3, label='tau = -3')



# Add labels and legend
#ax.xlabel('Index')
#ax.ylabel('Values')
#ax.title('FEPLS Estimation of beta - Confidence intervals')
plt.legend()
#plt.savefig('beta_estim_corr_2_0_9.pdf')
# Show the plot
plt.show()



In [ ]:
N=500
n=500
d=101
s_Y=1 # scale parameter of Pareto/Lomax distribution
tau=-2 # tail index of \vfi
#nu = 1 #  tail index of \psi such that 2\gamma \nu_j < 1 for all 1\le j \le J
snr=10 # signal-to-noise ratio
H=1/3 # Hurst parameter of fBm noise
rho=-1/2
s = 3/4 # tunes the threshold y_n (see below)
l=2 # grid parameter
#y=n**(s*gamma) # threshold y_n
#y_matrix = y*np.ones((N,n)) # threshold matrix
mu = 200 # noise mean
#theta=1/np.sqrt(gamma) # first parameter of Burr distribution 
#rho=1/np.sqrt(gamma) # second parameter of Burr distribution
#theta=1 # first parameter of Burr distribution 
#rho=1/gamma # second parameter of Burr distribution


c= 2 # c is \kappa the tail index of g
gamma=9/10# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_2_0_9.pdf')  
plt.show()

c= 2 # c is \kappa the tail index of g
gamma=1/2# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_2_0_5.pdf')  
plt.show()

c= 2 # c is \kappa the tail index of g
gamma=1/3# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_2_0_333.pdf')  
plt.show()

c= 1 # c is \kappa the tail index of g
gamma=9/10# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_1_0_9.pdf')  
plt.show()

c= 1 # c is \kappa the tail index of g
gamma=1/2# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_1_0_5.pdf')  
plt.show()



c= 1 # c is \kappa the tail index of g
gamma=1/3# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_1_0_333.pdf')  
plt.show()

c= 1 # c is \kappa the tail index of g
gamma=9/10# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_1_0_9.pdf')  
plt.show()

c= 3/2 # c is \kappa the tail index of g
gamma=1/2# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_1_5_0_5.pdf')  
plt.show()

c= 3/2 # c is \kappa the tail index of g
gamma=1/3# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix, bins=m)
plt.savefig('hist_k_conc_1_5_0_333.pdf')  
plt.show()


c= 3/2 # c is \kappa the tail index of g
gamma=9/10# 1/3 or 1/2 or 9/10
#Y=Pareto_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
#Y=Lomax_quantile_function(npr.uniform(0,1,size=(N,n)),theta,s_Y)
Y=Burr_quantile_function(npr.uniform(0,1,size=(N,n)),gamma,rho)
Y_sort_index = np.argsort(Y,axis=1)
Z=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux=np.multiply.outer(Y**c,beta_func(d)) # g(Y)\beta
eps=coeurjolly_cholesky_fbm_var(Y,Z,H,c,snr,mu) 
X=aux+eps
m=int(n/5)
y_matrix = threshold_index(X,Y,Y_sort_index,tau,m)
plt.hist(y_matrix,m)
plt.savefig('hist_k_conc_1_5_0_9.pdf')  
plt.show()

In [ ]:
################################################## Conditional quantile estimation

@numba.njit(parallel=True, fastmath=False) 
def Epanechnikov_kernel(x): # x is a np.array of shape (p,q);     W=np.where(np.abs(x)<=1,1,0)
    out = np.zeros_like(x)
    x=np.asarray(x)
    for i in numba.prange(x.shape[0]):
        for j in numba.prange(x.shape[1]):
            if x[i,j]<=1 and x[i,j]>=0:
                out[i,j] = np.multiply(3/2,1-np.power(x[i,j],2))
            else:
                out[i,j]=0
    return out
    
@numba.njit(parallel=False, fastmath=False) 
def Gaussian_kernel(x):
    return (1/np.sqrt(2*np.pi))*np.exp(-0.5*np.power(x,2))

@numba.njit(parallel=True, fastmath=False) 
def univariate_Nadaraya_weight(X_dimred,x,h_vect,c,gamma,rho): # X_dimred of shape (N,n) just as Y and X_dimred = hatbeta_dot_X(X,hat_beta,y_matrix,tau)
    # hat_beta=fepls(X,Y,y_matrix,tau)
    # h_vect.shape = (N,); c is kappa 
    # x in (0,1)
    N=X_dimred.shape[0]
    n=X_dimred.shape[1]
    out=np.zeros((N,n))
    for k in numba.prange(N):
        K_h=Gaussian_kernel((X_dimred[k,:]-Burr_quantile_function(x,gamma,rho)**c)/h_vect[k]) # shape (N,)
        out[k,:]=K_h/np.sum(K_h) 
    return out ### shape = (N,n)

@numba.njit(parallel=True, fastmath=False) 
def functional_Nadaraya_weight(X,x,h_vect,c,gamma,rho): # X.shape = (N,n,d); h_vect.shape = (N,); x in (0,1)
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]
    X_fake = (Burr_quantile_function(x,gamma,rho)**c)*beta_func(d) # shape is (d,)   
    aux = np.zeros((N,n,d))
    for p in numba.prange(d):
        aux[:,:,p]=(X[:,:,p]-X_fake[p])**2
    norm = np.sqrt((1/d)*np.sum(aux,axis=2))
    out=np.zeros((N,n))
    K_h=Epanechnikov_kernel(np.divide(norm,h_vect.reshape(-1,1)))
    for k in numba.prange(N):
        out[k,:]=K_h[k,:]/np.sum(K_h[k,:]) 
    return out ### shape = (N,n)

@numba.njit(parallel=True, fastmath=False)   
def weighted_quantile(data,weight,alpha):  # data.shape=weight.shape=(n,) 
    # alpha is the treshold in (0,1)
    sorter = np.argsort(data)
    data = data[sorter]
    weight = weight[sorter]
    weighted_quantiles = np.cumsum(weight) - 0.5 * weight
    weighted_quantiles /= np.sum(weight)
    return np.interp(alpha, weighted_quantiles, data)

@numba.njit(parallel=True, fastmath=False) 
def iterated_weq(Y,weight,alpha): # same treshold for all marginals and iterations # weq = weighted empirical quantile
    # data.shape = (N,n); weight.shape=(N,n)
    N=Y.shape[0]
    n=Y.shape[1]
    out = np.zeros((N,))
    for k in numba.prange(N):
        out[k]=weighted_quantile(Y[k,:],weight[k,:],alpha)
    return out # shape = (N,)

@numba.njit(parallel=True, fastmath=False) 
def tail_index_gamma_estimator(Y,weight,alpha,J): 
    N=Y.shape[0]
    subdivision=np.array([(1/s) for s in np.arange(1,J+1)] )
    quantile_data2=iterated_weq(Y,weight,alpha) 
    out=np.zeros((N,))
    aux=np.zeros((N,J))
    for k in numba.prange(N):
        for j in numba.prange(J):
            quantile_data1=iterated_weq(Y,weight,1-subdivision[j]*(1-alpha))
            aux[k,j] = np.log(quantile_data1[k])-np.log(quantile_data2[k])
            aux[k,j] /= -np.sum(np.log(subdivision))
        out[k] = np.sum(aux[k,:])
    return out

@numba.njit(parallel=True, fastmath=False)  
def plot_tail_index_func(X,Y,h_vect,c,gamma,rho,alpha,J): #h_vect=h*np.ones((N,)); h=2*mu; J=9; alpha in (0,1)
    N=X.shape[0]
    n=X.shape[1]
    d=X.shape[2]
    x_grid = np.linspace(0,1,d)
    out=np.zeros((N,d))
    for k in numba.prange(d):
        weight=functional_Nadaraya_weight(X,x_grid[k],h_vect,c,gamma,rho)
        #weight=univariate_Nadaraya_weight(X,x_grid[k],h_vect,c,gamma,rho)            
        out[:,k]=tail_index_gamma_estimator(Y,weight,alpha,J)
    return out

@numba.njit(parallel=True, fastmath=False)  
def plot_tail_index_uni(X_dimred,Y,h_vect,c,gamma,rho,alpha,J): #h_vect=h*np.ones((N,)); h=2*mu; J=9; alpha in (0,1); c is kappa
    N=X_dimred.shape[0]
    n=X_dimred.shape[1]
    x_grid = np.linspace(0,1,d)
    out=np.zeros((N,d))
    for k in numba.prange(d):
        weight=univariate_Nadaraya_weight(X_dimred,x_grid[k],h_vect,c,gamma,rho)          
        out[:,k]=tail_index_gamma_estimator(Y,weight,alpha,J)
    return out


In [ ]:
######################################## Comparison tail index estimation with X and with <X,hat_beta>
########################### New model for X and Y with latent variable Z uniform on [0,3^{-1/2}]
##################### X=g(Y)\beta+\eps and Y = Burr_distribution with \gamma(Z) = 2/3+Z^2  
##### Still Draft


@numba.njit(parallel=True, fastmath=False) 
def univariate_Nadaraya_weight_new(X_dimred_new,x,z,h_vect,c,rho): # X_dimred of shape (N,n) just as Y and X_dimred = hatbeta_dot_X(X,hat_beta,y_matrix,tau)
    # hat_beta=fepls(X,Y,y_matrix,tau)
    # h_vect.shape = (N,); c is kappa 
    # x in (0,1), z in [0,3^{-1/2}]
    N=X_dimred_new.shape[0]
    n=X_dimred_new.shape[1]
    out=np.zeros((N,n))
    for k in numba.prange(N):
        K_h=Gaussian_kernel((X_dimred_new[k,:]-Burr_quantile_function(x,2/3+z**2,rho)**c)/h_vect[k]) # shape (N,)
        out[k,:]=K_h/np.sum(K_h) 
    return out ### shape = (N,n)

@numba.njit(parallel=True, fastmath=False) 
def functional_Nadaraya_weight_new(X_new,x,z,h_vect,c,rho): # X.shape = (N,n,d); h_vect.shape = (N,); x in (0,1)
    N=X_new.shape[0]
    n=X_new.shape[1]
    d=X_new.shape[2]
    X_fake = (Burr_quantile_function(x,2/3+z**2,rho)**c)*beta_func(d) # shape is (d,)   
    aux = np.zeros((N,n,d))
    for p in numba.prange(d):
        aux[:,:,p]=(X_new[:,:,p]-X_fake[p])**2
    norm = np.sqrt((1/d)*np.sum(aux,axis=2))
    out=np.zeros((N,n))
    K_h=Epanechnikov_kernel(np.divide(norm,h_vect.reshape(-1,1)))
    for k in numba.prange(N):
        out[k,:]=K_h[k,:]/np.sum(K_h[k,:]) 
    return out ### shape = (N,n)

@numba.njit(parallel=True, fastmath=False)  
def plot_tail_index_func_new(X_new,Y_new,h_vect,c,rho,alpha,J): #h_vect=h*np.ones((N,)); h=2*mu; J=9; alpha in (0,1)
    N=X_new.shape[0]
    n=X_new.shape[1]
    d=X_new.shape[2]
    z_grid = np.linspace(0,3**(-1/2),d)
    out=np.zeros((N,d))
    for k in numba.prange(d):
        weight=functional_Nadaraya_weight_new(X_new,x_grid[k],h_vect,z,c,rho)
        out[:,k]=tail_index_gamma_estimator(Y_new,weight,alpha,J)
    return out

@numba.njit(parallel=True, fastmath=False)  
def plot_tail_index_uni_new(X_dimred_new,Y_new,h_vect,c,rho,alpha,J): #h_vect=h*np.ones((N,)); h=2*mu; J=9; alpha in (0,1); c is kappa
    N=X_dimred_new.shape[0]
    n=X_dimred_new.shape[1]
    z_grid = np.linspace(0,3**(-1/2),d)
    out=np.zeros((N,d))
    for k in numba.prange(d):
        weight= univariate_Nadaraya_weight_new(X_dimred_new,x_grid[k],z,h_vect,c,rho)
        out[:,k]=tail_index_gamma_estimator(Y_new,weight,alpha,J)
    return out

alpha=0.765
J=9
h=2*mu
h_vect=h*np.ones((N,))

def tail_index_of_Y(z):
    return 2/3 + z**2

Z_new = npr.uniform(0,3**(-1/2),size=(N,n)) # latent variables for both Y
Y_new = Burr_quantile_function(npr.uniform(0,1,size=(N,n)),tail_index_of_Y(Z_new),rho)
L_new=npr.normal(0,1,size=(N,n,d - 1)) # latent variables for fBm sampling
aux_new=np.multiply.outer(Y_new**c,beta_func(d)) # g(Y)\beta
eps_new=coeurjolly_cholesky_fbm_var(Y_new,L_new,H,c,snr,mu) 
X_new=aux_new+eps_new

alpha=0.765
J=9
h=2*mu
h_vect=h*np.ones((N,))

plot1=plot_tail_index_func_new(X_new,Y_new,h_vect,c,rho,alpha,J)
hat_beta_new=fepls(X_new,Y_new,y_matrix,tau)
X_dimred_new = hatbeta_dot_X(X_new,hat_beta_new,y_matrix,tau)
plot2=plot_tail_index_uni_new(X_dimred_new,Y_new,h_vect,c,rho,alpha,J)


max_values1 = np.nanquantile(plot1, 0.95, axis=0) # shape (d,)
min_values1 = np.nanquantile(plot1, 0.05, axis=0) # shape (d,)
mean_values1 = np.nanmean(plot1, axis=0) # shape (d,)
median_values1 = np.nanmedian(plot1, axis=0) # shape (d,)

max_values2 = np.nanquantile(plot2, 0.95, axis=0) # shape (d,)
min_values2 = np.nanquantile(plot2, 0.05, axis=0) # shape (d,)
mean_values2 = np.nanmean(plot2, axis=0) # shape (d,)
median_values2 = np.nanmedian(plot2, axis=0) # shape (d,)

# Create x values (assuming x values are just indices in this case)
x_values = np.arange(d)

fig,ax = plt.subplots()

# Set x-axis tick labels to discretized interval [0,1]
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
ax.set_xticklabels([0,0 , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1 ])

# Plot the fill between the max and min values
ax.fill_between(x_values, min_values1, max_values1, color='skyblue', alpha=0.4)
ax.plot(x_values, mean_values1)
ax.fill_between(x_values, min_values2, max_values2, color='blue', alpha=0.2)
ax.plot(x_values, mean_values2)

plt.show()